## Problem formulation

## Basic ARMA model

Basic ARMA(p, q) model can be presented as:
    \begin{equation}
    x_t = \mu + \varepsilon_t +  \sum_{i=1}^p \phi_i x_{t-i} + \sum_{i=1}^q \theta_i e_{t-i}
    \end{equation}
where $\varepsilon_i$ are error terms which are expected to form a white noise process (i.e. are independent identically distributed random variables with $\varepsilon_i \sim N(0, \sigma^2)$).
The key obstacle in analysing and fitting an ARMA model is that error terms $\varepsilon_i$ are not observable, nor can they be simply calculated as it is in case of, for example, linear regression ($\varepsilon = Y - \alpha - \beta X$). The equation above 
    \begin{equation}
    \varepsilon_t = x_t - (\mu + \sum_{i=1}^p \phi_i x_{t-i} + \sum_{i=1}^q \theta_i e_{t-i})
    \end{equation}
essencially makes $\varepsilon_t$ a function of $\{\varepsilon_{t-1}...\varepsilon_{t-p}\}$ which, in turn, can only be calculated recursivelly through previous values of $\varepsilon$.

## Likelihood function

In order to fit the model to available data $X_n = \{X_1, ..., X_n\}$ and find optimal parameters $\mu, \sigma, \phi,  \theta$ we need to maximize the likelihood function, which means to find a set of parameters, for which the joint probability of $\{X_1, ..., X_n\}$ is the highest:
    \begin{equation}
    L = f(X_n | \mu, \sigma, \phi,  \theta)
    \end{equation}
which is, from what we know of conditional probabilities, equivalent to:
    \begin{equation}
    L = \prod_{i=1}^n f(X_i | X_{i-1}, \mu, \sigma, \phi,  \theta)
    \end{equation}
or
    \begin{equation}
    l = \ln(L) = \sum_{i=1}^n f(X_i | X_{i-1}, \mu, \sigma, \phi,  \theta)
    \end{equation}
If we assume that the errors $\varepsilon$ are Gaussian then $x_t$ are also expected to be Gaussian as a linear combination of $\varepsilon$, $x_0$ and some fixed model parameters. Which means that:
    \begin{equation}
    l = \sum_{i=1}^n -\frac{1}{2} \big( \ln(2\pi|F_i|) + \nu_i F_i^{-1}\nu_i \big) 
    \end{equation}
where
    \begin{equation}
    \nu_t = x_t - E(x_t|x_1...x_{t-1}) \quad and \quad F_t = E(\nu_t \nu_t' |x_1...x_{t-1}).
    \end{equation}

# State space representation



The problem can be reformulated in matrix form using a state space representation with $m$-dimensional vectors $\alpha_t$ such that:
    \begin{equation}
    \alpha_t^i = \phi_i x_{t-1} + \alpha_{t-1}^{i+1} + \theta_{i-1} \varepsilon_t
    \end{equation}
where $m = \max(p, q+1)$ and where the coefficients $\phi_i$ for $p<i\le m$ and $\theta_i$ for $q<i\le m$ are assumed to be $0$ and $\theta_0 = 1$.

It allows us to split our system into **transition equation** which can be interpreted as its true state under assumed ARMA(p, q) mode:
    \begin{equation}
    \alpha_t = K\alpha_{t-1} + \varepsilon_t* R 
    \end{equation}
where:
    \begin{equation}
    K=
      \begin{bmatrix}
        \phi_1 & 1 & 0 & ... & 0 \\
        \phi_2 & 0 & 1 & ... & 0 \\
        \vdots & \vdots & \vdots & ...  & \vdots \\ 
        \phi_m & 0 & 0 & ... & 0
      \end{bmatrix}
       \quad and \quad
     R=
      \begin{bmatrix}
        1\\
        \theta_1\\
        ...  \\ 
        \theta_m
      \end{bmatrix}
    \end{equation}
  and **measurement equation**, where 
      \begin{equation}
    \alpha_t^1 = \phi_1 x_{t-1} + \alpha_{t-1}^1 + \varepsilon_t =
        \phi_1 x_{t-1} + \phi_2 x_{t-2} + \alpha_{t-2}^1 + \theta_1 \varepsilon_{t-1} +  \varepsilon_t = ... =
        x_t - \mu
    \end{equation}
  or $x_t = Z \alpha_t + \mu$ where $Z = \begin{bmatrix}1 & 0 & ... & 0 \end{bmatrix}$.
    

## Kalman filter

Kalman filter is a recursive algorithm which allows us to forecast the distribution of $x_t | x_1...x_{t-1}$ so that we can calculate $\nu_t$ and $F_t$ and the likelihood function. The algorithm works with $\alpha_t$ and consists of two steps:
- predicting the estimate for the next $\alpha_t$ and its correlation matrix $P_t$ :
    \begin{equation}
    \hat{\alpha_t} = K\alpha_{t-1} \\
    \hat{P_t} = KP_{t-1}K' + RQR' \\
    \hat{x_t} = Z\hat{a_t} + \mu
    \end{equation}
- updating the values of $\alpha_t$ and $P_t$ based on actual $x_t$:
    \begin{equation}
    \Gamma_t = \hat{P_t} Z' F_t^{-1} \\
    P_{t} = \hat{P_t} - \Gamma_t Z  \hat{P_t}'\\
    \alpha_t = \hat{\alpha_t} +  \Gamma_t \nu_t
    \end{equation}

$\nu$ and $F$ obtained via Kalman filter procedure
    \begin{equation}
    \nu_t = x_t - \hat{x_t} \\
    F_t = Z \hat{P_t} Z' \\
    \end{equation}
allow us to calculate $l$ and thus to minimize it using one of the numerical minimization algorithms.

In [ ]:
class KalmanFilter:

    def __init__(self, mu, sigma, phi, theta):
        self.phi = phi
        self.theta = theta
        self.mu = mu
        self.sigma = sigma
        m = len(phi)
        R = np.concatenate((np.ones(1), self.theta[:-1]))
        self.R = np.diag(R)
        K = np.concatenate((self.phi[:-1].reshape((-1, 1)), np.identity(m - 1)), axis=1)
        self.K = np.concatenate((K, np.concatenate((self.phi[-1:], np.zeros(m - 1))).reshape(1, -1)))
        self.Q = sigma ** 2 * np.identity(m)
        self.Z = np.zeros((1, m))
        self.Z[0, 0] = 1.0

    def update(self, a_hat, p_hat, F, nu):
        gain = multi_dot([p_hat, np.transpose(self.Z), np.linalg.inv(F)])
        a = a_hat + multi_dot([gain, nu])
        p = p_hat - multi_dot([gain, self.Z, np.transpose(p_hat)])
        return a, p

    def predict(self, a, p, x):
        a_hat = np.matmul(self.K, a)
        p_hat = mat_square(p, self.K) + mat_square(self.Q, self.R)
        x_hat = np.matmul(self.Z, a_hat) + self.mu
        F = mat_square(p_hat, self.Z)
        nu = x - x_hat
        return a_hat, p_hat, x_hat, F, nu

In [ ]:
def _likelihood(X, mu, sigma, phi, theta, errors=False):

    loglikelihood = 0.0
    m = phi.size
    kalman = KalmanFilter(mu, sigma, phi, theta)
    p = np.identity(m)
    a = np.zeros((m, 1))
    eps = np.zeros(len(X))
    for i, x in enumerate(X):
        a_hat, p_hat, x_hat, F, nu = kalman.predict(a, p, x)
        LL_last = -0.5 * (np.log(2 * np.pi * np.abs(F)) + mat_square(np.linalg.inv(F), nu))
        a, p = kalman.update(a_hat, p_hat, F, nu)
        eps[i] = nu
        loglikelihood += LL_last
    if errors:
        return eps
    else:
        return -float(loglikelihood)

## Initial conditions

As it is shown in (9) initial conditions of Kalman filter process for atationary ARMA model can be estimated as:
\begin{equation}
\alpha_1 \sim N(0, \sigma^2 Q_0) 
\end{equation}
where $Q_0$ is obtained from $m^2$-dimensional equation:
\begin{equation}
vec(Q_0) = vec(RR') (I_{m^2} - K \otimes K)^{-1}
\end{equation}
where $\otimes$ is Kronecker product and $vec(x)$ is a stacked vector of columns of $x$. It means that
\begin{equation}
\alpha_1 = \begin{bmatrix}
0 \\ ... \\ 0 
\end{bmatrix} 
\quad and \quad
P_1 = \sigma^2 Q_0.
\end{equation}

## Conditional sum of squares (CSS)

Conditional sum of squares is a simplification of state space representation with an assumption that initial unobserved value of $\alpha_0$ is known (for example $\alpha_0 = 0$). In this case instead of estimating both expected value and covariance matrix of $\alpha_t$ we can perform explicit calculations:
\begin{equation}
\varepsilon^*_t = x_t - Z\hat{\alpha_t} - \mu \quad \text{where} \quad \hat{\alpha_t} = K \hat{\alpha_{t-1}} + K R \varepsilon^*_{t-1}
\end{equation}

and thus:
    \begin{equation}
    l = \sum_{i=1}^n -\frac{1}{2} \big( \ln(2\pi\sigma^2) + \frac{\varepsilon_i^* \varepsilon_i^*}{\sigma^2} \big)
      =  -\frac{1}{2} \big( n \ln(2\pi\sigma^2) + \frac{1}{\sigma^2} \sum_{i=1}^n \varepsilon_i^{*2} \big)
    \end{equation}

which is a much simple model yet it is proven to asymptotically converge to MLE estimates[6].
Recursive error calculation can be represented as an IIR filter[7] where $x_t$ is an input signal and $\varepsilon^*_t$ is output signal. It allows us to use ``lfilter()`` from ``scipy.signal`` package which performs such calculation effeciently.

In [ ]:
def _run_css(params, X, len_p, errors=False, transform=True):

    mu = params[0]
    nobs = len(X) - len_p
    if transform:
        phi = np.r_[1, -_transform_params(params[2:len_p + 2])]
        theta = np.r_[1, _transform_params(params[len_p + 2:])]
    else:
        phi = np.r_[1, -params[2:len_p + 2]]
        theta = np.r_[1, params[len_p + 2:]]

    y = X - mu
    init = np.zeros((max(len(phi), len(theta)) - 1))
    for i in range(len_p):
        init[i] = sum(-phi[:i + 1][::-1] * y[:i + 1])
    eps = lfilter(phi, theta, y, zi=init)[0][len_p:]
    if errors:
        return eps
    else:
        ssr = np.dot(eps, eps)
        sigma2 = ssr / nobs
        loglikelihood = -nobs / 2. * (np.log(2 * np.pi * sigma2)) - ssr / (2. * sigma2)
        return -loglikelihood

## Stationarity and invertability

Arima model is stationary if absolute value of all the roots of the polynom:
\begin{equation}
    \phi(z) = 1 - \sum_{i=1}^{p}\phi_i z^i
\end{equation}
is greater than 1.

Arima model is invertible (i.e. can be converted to an AR($\infty$) model)  if absolute value of all the roots of the polynom:
\begin{equation}
    \theta(z) = 1 + \sum_{i=1}^{p}\theta_i z^i
\end{equation}
is greater than 1.

*** Add about transformation

## Initialization of $\phi$ and $\theta$

A simple estimate of ARMA coefficients typically improves the convergence speed against random or zero-valued initialization, such as the following scheme:
- first, AR-coefficients $(\phi')$ are estimated via simple linear regression and OLS:
\begin{equation}
 x_t = \mu + \sum_{i=1}^p \phi'_i x_{t-i} + r_t
\end{equation}
- then the residuals $r_t$ can be used for a second linear regression, this time with MA components $\theta'$:
\begin{equation}
 x_t = \mu + \sum_{i=1}^p \phi'_i x_{t-i} + \sum_{i=1}^q \theta'_i r_{t-i} + \varepsilon_t 
\end{equation}
the obtained parameters $\phi'$ and $\theta'$ can be used as an initial value in MLE or CSS optimization process.

### Materials:
 1. https://otexts.com/fpp2/non-seasonal-arima.html - basic problem formulation
 2. https://uh.edu/~bsorense/kalman.pdf - Kalman filter for ARIMA case
 3. https://github.com/rlabbe/Kalman-and-Bayesian-Filters-in-Python - general information about filters
 4. https://www.nuffield.ox.ac.uk/economics/Papers/1997/w6/ma.pdf - relationship between CSS and MLE
 5. https://en.wikipedia.org/wiki/Infinite_impulse_response - general information on IIR
 6. http://www-stat.wharton.upenn.edu/~stine/stat910/lectures/08_intro_arma.pdf - general ARIMA information + stationarity conditions
 7. Time Series Analysis by State Space Methods by J. Durbin and S.J. Koopman 
 8. https://www.jstor.org/stable/2335856 - on model coefficients initialization